## Enzyme Kinetics Analysis


You are studying enzyme activity of fumarase. Fumarase is a tetrameric enzyme that catalyzes the conversion of fumarate to malate. 

![fumarase](1fuo2.jpg) (https://commons.wikimedia.org/wiki/File:1fuo2.jpg)
![reaction](Reaction1.png) (https://en.wikipedia.org/wiki/Fumarase)

You are using the the non-natural substrate chlorofumarate: ![chlorofumarate](chlorofumarate.png)  This molecule absorbs strongly in the UV, but the reaction product does not, giving a spectroscopic signal you can observe to follow reaction progress.  

## Tasks

You have Product vs. Time data for both wildtype fumarase and a mutant form of the enzyme. These were done at a variety of substrate concentrations, given in micromolar in the data filenames (e.g. S=200 means $[S]_{0} = 200 \mu M$. All experimental output can be found in the `data` directory. 
+ Create P vs. t plots for each experiment and extract $k$.
+ Use this information to determine $V_{0}$ for each $[S]_{0}$.
+ Construct $[S]_{0}$ vs. $V_{0}$ curves and extract Michaelis-Menten parameters.
+ Determine the effect of the mutation on these values.
+ Come up with a hypothesis for what the mutation might be doing to have the observed effect.

Run the five cells below, in order, to get started. Read through the second cell to have a basic understanding of what models are defined.

In [ ]:
# import libraries
import numpy as np
import scipy.optimize as optimize
import scipy.stats as stats
import matplotlib.pylab as plt
import pandas as pd
import glob
%matplotlib inline

In [ ]:
# Define functions for fitting models

# ------- First order kinetics equations ----------

def first_order(t,k,S0):
    return S0*(1 - np.exp(-k*t))
    
def first_order_residuals(k,S0,t,obs):
    return obs - first_order(t,k,S0)

def fit_first_order(filename):

    d = pd.read_csv(filename)
    S0 = float(filename.split("=")[1].split(".")[0])
    
    k_guess = 0.1
    
    fit_param, covariance, info_dict, message, error = optimize.leastsq(first_order_residuals,
                                                                        x0=np.array((k_guess)),
                                                                        args=(S0,d.t,d.P),full_output=True)
    
    print("S0 = ",S0)
    print("k = ",fit_param[0])
    #print("V = ",fit_param[0]*S0)
    
    plt.plot(d.t,d.P,"o")
    plt.plot(d.t,first_order(d.t,fit_param[0],S0))
    plt.title(filename)
    plt.xlabel("t (s)")
    plt.ylabel("product (uM)")
    plt.show()
    
    #return fit_param[0]*S0

# --------- Michaelis-Menten equation -----------------

def mm(S,kcat,Km,Et=5):    
    return kcat*Et*(S/(S + Km))

def mm_residuals(param,S,obs):
    
    kcat = param[0]
    Km = param[1]
    return obs - mm(S,kcat,Km)
    
def fit_mm(S,V):
    
    kcat_guess = 10
    Km_guess = 100
    
    S = np.array(S)
    V = np.array(V)
    
    fit_param, covariance, info_dict, message, error = optimize.leastsq(mm_residuals,
                                                                        x0=np.array((kcat_guess,Km_guess)),
                                                                        args=(S,V),full_output=True)
    print("kcat = ",fit_param[0])
    print("Km = ",fit_param[1])
    
    fit_S = np.arange(S[0],S[-1])
    
    plt.plot(S,V,"o")
    plt.plot(fit_S,mm(fit_S,fit_param[0],fit_param[1]))
    plt.title("MM fit")
    plt.xlabel("[S]0 (uM)")
    plt.ylabel("V0 (uM/s)")
    plt.show()

# ------------ Combined Hill and Michaelis-Menten euqations ---------------
    
def mm_hill(S,kcat,Km,n,Et=5):
    return kcat*Et*(S**n/(S**n + Km**n))

def mm_hill_residuals(param,S,obs):
    
    kcat = param[0]
    Km = param[1]
    n = param[2]
    
    return obs - mm_hill(S,kcat,Km,n)
    
def fit_mm_hill(S,V):
    
    kcat_guess = .03
    Km_guess = 10.
    n_guess = 3
    
    S = np.array(S)
    V = np.array(V)
    
    fit_param, covariance, info_dict, message, error = optimize.leastsq(mm_hill_residuals,
                                                                        x0=np.array((kcat_guess,Km_guess,n_guess)),
                                                                        args=(S,V),full_output=True)
    print("kcat = ",fit_param[0])
    print("Km = ",fit_param[1])
    print("n = ",fit_param[2])
        
    fit_S = np.arange(S[0],S[-1])
    
    plt.plot(S,V,"o")
    plt.plot(fit_S,mm_hill(fit_S,fit_param[0],fit_param[1],fit_param[2]))
    plt.title("MM fit")
    plt.xlabel("[S]0 (uM)")
    plt.ylabel("V0 (uM/s)")
    plt.show()   
    

In [ ]:
# Fit to wildtype P vs. t data at [S] = 100 uM
V = fit_first_order("data/wt_cf_S=100.txt")

In [ ]:
# Dummy fit of made-up [S] vs. [V] data
fit_mm([0,1,2,3,10],[0,10,15,17,22])

In [ ]:
# Dummy fit of made-up [S] vs. [V] data
fit_mm_hill([0,1,2,3,10],[0,10,15,17,22])